In [ ]:
!pip install --upgrade git+https://github.com/zalandoresearch/flair.git

  Cloning https://github.com/zalandoresearch/flair.git to /tmp/pip-req-build-e0zwgcdn
  Running command git clone --filter=blob:none --quiet https://github.com/zalandoresearch/flair.git /tmp/pip-req-build-e0zwgcdn
  Resolved https://github.com/zalandoresearch/flair.git to commit b034b8cef1509448a1280b861e78d09f541717b8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 77.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.2/201.2 kB 27.9 MB/s eta 0:00:00
  Preparing

In [ ]:
!pip install transformers

In [ ]:
from flair.data import Corpus
from flair.datasets import SentenceDataset
from flair.trainers import ModelTrainer
from flair.models import TARSClassifier
from flair.data import Sentence, Label

import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import re
import copy
from tqdm.notebook import tqdm
import gc

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader

project_dir = './'

In [ ]:
df_train = pd.read_csv(project_dir + 'train_data.csv')
print(df_train.shape)
df_train.head()

(1387, 16)


,ID,Text,Reconnaissance,Resource Development,Initial Access,Execution,Persistence,Privilege Escalation,Defense Evasion,Credential Access,Discovery,Lateral Movement,Collection,Command and Control,Exfiltration,Impact
0,CVE-2020-15100,"In freewvs before 0.1.1, a user could create a...",0,0,1,0,1,0,0,0,0,0,0,0,0,1
1,CVE-2017-0262,"Microsoft Office 2010 SP2, Office 2013 SP1, an...",0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,CVE-2021-33709,A vulnerability has been identified in Teamcen...,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3,CVE-2021-3393,An information leak was discovered in postgres...,0,0,0,1,0,0,0,0,0,0,1,0,0,0
4,CVE-2022-26850,When creating or updating credentials for sing...,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [ ]:
df_test = pd.read_csv(project_dir + 'test_data.csv')
print(df_test.shape)
df_test.head()

(331, 16)


,ID,Text,Reconnaissance,Resource Development,Initial Access,Execution,Persistence,Privilege Escalation,Defense Evasion,Credential Access,Discovery,Lateral Movement,Collection,Command and Control,Exfiltration,Impact
0,CVE-2021-34803,TeamViewer before 14.7.48644 on Windows loads ...,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,CVE-2016-0034,Microsoft Silverlight 5 before 5.1.41212.0 mis...,0,0,1,1,0,0,0,0,0,0,0,0,0,1
2,CVE-2015-5123,Use-after-free vulnerability in the BitmapData...,0,0,1,1,0,0,0,0,0,0,0,0,0,0
3,CVE-2021-23964,Mozilla developers reported memory safety bugs...,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,CVE-2020-5812,Nessus AMI versions 8.12.0 and earlier were fo...,0,0,1,0,0,0,0,1,0,0,1,0,0,0


In [ ]:
df_test.shape

(331, 16)

In [ ]:
df_test.head()

,ID,Text,Reconnaissance,Resource Development,Initial Access,Execution,Persistence,Privilege Escalation,Defense Evasion,Credential Access,Discovery,Lateral Movement,Collection,Command and Control,Exfiltration,Impact
0,CVE-2021-34803,TeamViewer before 14.7.48644 on Windows loads ...,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,CVE-2016-0034,Microsoft Silverlight 5 before 5.1.41212.0 mis...,0,0,1,1,0,0,0,0,0,0,0,0,0,1
2,CVE-2015-5123,Use-after-free vulnerability in the BitmapData...,0,0,1,1,0,0,0,0,0,0,0,0,0,0
3,CVE-2021-23964,Mozilla developers reported memory safety bugs...,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,CVE-2020-5812,Nessus AMI versions 8.12.0 and earlier were fo...,0,0,1,0,0,0,0,1,0,0,1,0,0,0


In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

final_texts = []
final_labels = []

def clean_abstract(text):
    # Replace versions with the word "VERSION"
    version_pattern = r"\d+(\.\d+)+"
    updated_text = re.sub(version_pattern, "version", text)

    # Replace other CVE references
    version_pattern = r'CVE-\d{1,5}-\d{1,5}'
    updated_text = re.sub(version_pattern, "CVE", updated_text)

    return updated_text

def get_texts(df):
    texts = df['Text'].apply(clean_abstract)
    texts = texts.values.tolist()
    return texts

def get_labels(df):
    labels_li = [' '.join(x.lower().split()) for x in df.columns.to_list()[2:]]
    labels_matrix = np.array([labels_li] * len(df))

    mask = df.iloc[:, 2:].values.astype(bool)
    labels = []
    for l, m in zip(labels_matrix, mask):
        x = l[m]
        if len(x) > 0:
            labels.append(x.tolist())
        else:
            labels.append([])
    return labels

def get_ids(df):
    ids = df['ID'].values.tolist()
    return ids

train_texts = get_texts(df_train)
train_labels = get_labels(df_train)
train_ids = get_ids(df_train)

test_texts = get_texts(df_test)
test_labels = get_labels(df_test)
test_ids = get_ids(df_test)

for text, label, id in zip(train_texts[:5], train_labels[:5], train_ids):
    print(f'Text -\t{text}')
    print(f'ID -\t{id}')
    print(f'Labels -\t{label}')
    print()

Text -	In freewvs before version, a user could create a large file that freewvs will try to read, which will terminate a scan process. This has been patched in version.
ID -	CVE-2020-15100
Labels -	['initial access', 'persistence', 'impact']

Text -	Microsoft Office 2010 SP2, Office 2013 SP1, and Office 2016 allow a remote code execution vulnerability when the software fails to properly handle objects in memory, aka "Office Remote Code Execution Vulnerability". This CVE ID is unique from CVE and CVE.
ID -	CVE-2017-0262
Labels -	['execution']

Text -	A vulnerability has been identified in Teamcenter Active Workspace V4 (All versions < Vversion), Teamcenter Active Workspace Vversion (All versions < Vversion), Teamcenter Active Workspace Vversion (All versions < Vversion). By sending malformed requests, a remote attacker could leak an application token due to an error not properly handled by the system.
ID -	CVE-2021-33709
Labels -	['execution', 'credential access']

Text -	An information

In [ ]:
print(test_labels)
print(train_labels)

[['execution'], ['initial access', 'execution', 'impact'], ['initial access', 'execution'], ['execution'], ['initial access', 'credential access', 'collection'], ['initial access', 'defense evasion'], ['impact'], ['initial access', 'execution', 'persistence', 'impact'], ['defense evasion', 'credential access'], ['credential access'], ['execution', 'lateral movement', 'collection'], ['execution', 'persistence', 'impact'], ['defense evasion', 'credential access'], ['privilege escalation', 'impact'], ['execution', 'privilege escalation', 'credential access'], ['defense evasion', 'impact'], ['initial access', 'execution', 'privilege escalation'], ['initial access', 'execution'], ['credential access'], ['initial access'], ['initial access', 'execution'], ['initial access', 'execution'], ['initial access', 'execution'], ['execution', 'defense evasion'], ['privilege escalation', 'impact'], ['initial access', 'execution'], ['initial access', 'privilege escalation', 'lateral movement'], ['execu

In [ ]:
tars_train_dataset = []
train_size = len(train_texts)

for i in range(train_size):
  sent = Sentence(train_texts[i])

  for label in train_labels[i]:
    sent.add_label(value=label, typename='cybersecurity-attack-tactic')

  tars_train_dataset.append(sent)

print(tars_train_dataset)

[Sentence[33]: "In freewvs before version, a user could create a large file that freewvs will try to read, which will terminate a scan process. This has been patched in version." → initial access (1.0); persistence (1.0); impact (1.0), Sentence[48]: "Microsoft Office 2010 SP2, Office 2013 SP1, and Office 2016 allow a remote code execution vulnerability when the software fails to properly handle objects in memory, aka "Office Remote Code Execution Vulnerability". This CVE ID is unique from CVE and CVE." → execution (1.0), Sentence[63]: "A vulnerability has been identified in Teamcenter Active Workspace V4 (All versions < Vversion), Teamcenter Active Workspace Vversion (All versions < Vversion), Teamcenter Active Workspace Vversion (All versions < Vversion). By sending malformed requests, a remote attacker could leak an application token due to an error not properly handled by the system." → execution (1.0); credential access (1.0), Sentence[72]: "An information leak was discovered in po

In [ ]:
tars_test_dataset = []
test_size = len(test_texts)

for i in range(test_size):
  sent = Sentence(test_texts[i])

  for label in test_labels[i]:
    sent.add_label(value=label, typename='cybersecurity-attack-tactic')

  tars_test_dataset.append(sent)

print(tars_test_dataset)

[Sentence[12]: "TeamViewer before version on Windows loads untrusted DLLs in certain situations." → execution (1.0), Sentence[45]: "Microsoft Silverlight 5 before version mishandles negative offsets during decoding, which allows remote attackers to execute arbitrary code or cause a denial of service (object-header corruption) via a crafted web site, aka "Silverlight Runtime Remote Code Execution Vulnerability."" → initial access (1.0); execution (1.0); impact (1.0), Sentence[86]: "Use-after-free vulnerability in the BitmapData class in the ActionScript 3 (AS3) implementation in Adobe Flash Player 13.x through version on Windows and OS X, 14.x through version on Windows and OS X, 11.x through version on Linux, and 12.x through version on Linux Chrome installations allows remote attackers to execute arbitrary code or cause a denial of service (memory corruption) via crafted Flash content that overrides a valueOf function, as exploited in the wild in July 2015." → initial access (1.0); ex

In [ ]:
corpus = Corpus(train=tars_train_dataset, test=tars_test_dataset)

2023-10-25 12:26:00,903 No dev split found. Using 0% (i.e. 139 samples) of the train split as dev data


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
tars = TARSClassifier.load('tars-base')

tars.add_and_switch_to_new_task("vulnerability-tactic-prediction", label_type='cybersecurity-attack-tactic', label_dictionary=corpus.make_label_dictionary(label_type='cybersecurity-attack-tactic'))

trainer = ModelTrainer(tars, corpus)

trainer.train(base_path='resources/taggers/tactic',
              learning_rate=0.02,
              mini_batch_size=1,
              max_epochs=20,
              train_with_dev=True,
              main_evaluation_metric=("weighted avg", "f1-score")
              )

2023-10-25 12:26:02,302 https://nlp.informatik.hu-berlin.de/resources/models/tars-base/tars-base-v8.pt not found in cache, downloading to /tmp/tmplpagtmw4


100%|██████████| 418M/418M [00:49<00:00, 8.82MB/s]

2023-10-25 12:26:52,750 copying /tmp/tmplpagtmw4 to cache at /root/.flair/models/tars-base-v8.pt


2023-10-25 12:26:53,909 removing temp file /tmp/tmplpagtmw4


2023-10-25 12:27:07,343 TARS initialized without a task. You need to call .add_and_switch_to_new_task() before training this model
2023-10-25 12:27:07,479 Computing label dictionary. Progress:


...

2023-10-25 12:27:07,545 Dictionary created for label 'cybersecurity-attack-tactic' with 14 values: execution (seen 522 times), initial access (seen 459 times), impact (seen 292 times), privilege escalation (seen 255 times), collection (seen 196 times), credential access (seen 165 times), persistence (seen 141 times), defense evasion (seen 95 times), discovery (seen 53 times), lateral movement (seen 36 times), resource development (seen 30 times), reconnaissance (seen 20 times), exfiltration (seen 10 times), command and control (seen 5 times)
2023-10-25 12:27:07,553 ----------------------------------------------------------------------------------------------------
2023-10-25 12:27:07,557 Model: "TARSClassifier(
  (tars_model): TextClassifier(
    (embeddings): TransformerDocumentEmbeddings(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (toke

2023-10-25 12:27:23,271 epoch 1 - iter 138/1387 - loss 0.64987587 - time (sec): 12.90 - samples/sec: 59.15 - lr: 0.020000 - momentum: 0.000000
2023-10-25 12:27:36,242 epoch 1 - iter 276/1387 - loss 0.60216030 - time (sec): 25.87 - samples/sec: 57.44 - lr: 0.020000 - momentum: 0.000000
2023-10-25 12:27:49,940 epoch 1 - iter 414/1387 - loss 0.59352897 - time (sec): 39.57 - samples/sec: 55.27 - lr: 0.020000 - momentum: 0.000000
2023-10-25 12:28:03,393 epoch 1 - iter 552/1387 - loss 0.58063650 - time (sec): 53.02 - samples/sec: 55.39 - lr: 0.020000 - momentum: 0.000000
2023-10-25 12:28:18,417 epoch 1 - iter 690/1387 - loss 0.57854743 - time (sec): 68.05 - samples/sec: 54.29 - lr: 0.020000 - momentum: 0.000000
2023-10-25 12:28:33,976 epoch 1 - iter 828/1387 - loss 0.56763416 - time (sec): 83.60 - samples/sec: 53.19 - lr: 0.020000 - momentum: 0.000000
2023-10-25 12:28:48,447 epoch 1 - iter 966/1387 - loss 0.55996090 - time (sec): 98.07 - samples/sec: 53.14 - lr: 0.020000 - momentum: 0.000000

100%|██████████| 6/6 [00:35<00:00,  5.95s/it]

2023-10-25 13:15:24,818 
Results:
- F-score (micro) 0.5209
- F-score (macro) 0.3166
- Accuracy 0.142

By class:
                      precision    recall  f1-score   support

           execution     0.6757    0.6944    0.6849       144
      initial access     0.5342    0.6094    0.5693       128
privilege escalation     0.3780    0.4306    0.4026        72
              impact     0.7164    0.6000    0.6531        80
   credential access     0.3171    0.5652    0.4062        46
          collection     0.4906    0.4906    0.4906        53
         persistence     0.4800    0.3243    0.3871        37
     defense evasion     0.3030    0.3846    0.3390        26
           discovery     0.2273    0.3571    0.2778        14
    lateral movement     0.0000    0.0000    0.0000        10
resource development     0.0000    0.0000    0.0000         7
      reconnaissance     0.3333    0.1667    0.2222         6
        exfiltration     0.0000    0.0000    0.0000         3
 command and contro

{'test_score': 0.5195914234938989}